In [1]:
import os
import pandas as pd
from deta import Deta
from dotenv import load_dotenv
import pymsteams


load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

drive = deta.Drive("data")

myTeamsMessage = pymsteams.connectorcard(os.environ.get('TEAMS_WEBHOOK'))


In [2]:
current_df = pd.read_csv('https://drive.deta.sh/v1/b0x22rtxtdf/data/files/download?name=current.csv', storage_options={'X-API-Key':os.environ.get("DETA_PROJECT_KEY")})
current_df.ds = pd.to_datetime(current_df.ds)
current = current_df.head(1).iloc[0]
current

ds                     2023-12-01 22:00:00
INFLOW_STRETCHER                         5
Infl_Stretcher_cum                     152
INFLOW_AMBULATORY                        3
Infl_Ambulatory_cum                    119
Inflow_Total                             8
Inflow_Cum_Total                       271
INFLOW_AMBULANCES                        2
Infl_Ambulances_cum                     43
FLS                                      1
CUM_ADMREQ                              31
CUM_BA1                                 22
WAITINGADM                              35
TTStr                                  113
TRG_HALLWAY1                             3
TRG_HALLWAY_TBS                          3
reoriented_cum                          26
reoriented_cum_MD                        0
QTRACK1                                  5
RESUS                                    4
Pod_T                                   51
POD_GREEN                               18
POD_GREEN_TBS                            1
POD_YELLOW 

In [3]:
df = current_df.copy()

In [4]:
current_ds = current.ds
current_ds

Timestamp('2023-12-01 22:00:00')

In [5]:
current_ds - pd.Timedelta(hours=current_ds.hour)


Timestamp('2023-12-01 00:00:00')

In [6]:
yesterday_total_inflow = current_df.set_index('ds').loc[current.ds - pd.Timedelta(hours=current_ds.hour)].Inflow_Cum_Total
yesterday_total_inflow

248

In [7]:
yesterday_ambulances = current_df.set_index('ds').loc[current.ds - pd.Timedelta(hours=current_ds.hour)].Infl_Ambulances_cum
yesterday_ambulances

47

In [8]:
yesterday_reorientations = current_df.set_index('ds').loc[current.ds - pd.Timedelta(hours=current_ds.hour)].reoriented_cum
yesterday_reorientations

13

In [46]:
current_df.set_index('ds').loc[current.ds - pd.Timedelta(hours=current_ds.hour)]

INFLOW_STRETCHER         0
Infl_Stretcher_cum     118
INFLOW_AMBULATORY        2
Infl_Ambulatory_cum    145
Inflow_Total             2
Inflow_Cum_Total       263
INFLOW_AMBULANCES        0
Infl_Ambulances_cum     42
FLS                      0
CUM_ADMREQ              41
CUM_BA1                 30
WAITINGADM              47
TTStr                   89
TRG_HALLWAY1             0
TRG_HALLWAY_TBS          0
reoriented_cum          31
reoriented_cum_MD       17
QTRACK1                  1
RESUS                    5
Pod_T                   47
POD_GREEN               16
POD_GREEN_TBS            1
POD_YELLOW              14
POD_YELLOW_TBS           1
POD_ORANGE              17
POD_ORANGE_TBS           1
POD_CONS_MORE2H         20
POD_IMCONS_MORE4H        1
POD_XRAY_MORE2H          2
POD_CT_MORE2H            2
POST_POD1                5
VERTSTRET               11
RAZ_TBS                  0
RAZ_LAZYBOY              5
RAZ_WAITINGREZ          13
AMBVERT1                32
AMBVERTTBS              24
Q

In [9]:
current_df.set_index('ds').resample("D").max()

,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,CUM_ADMREQ,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
ds,,,,,,,,,,,,,,,,,,,,,
2023-11-30,13,153,12,93,25,246,3,47,0,38,...,31,25,7,0,11,1,3,9,22,16
2023-12-01,19,153,17,119,27,271,5,47,1,40,...,27,27,10,0,18,4,3,6,17,14


In [10]:
import datetime
yesterday = current_df.ds.max().date() - datetime.timedelta(days=1)
yesterday = pd.to_datetime(yesterday)

In [11]:
yesterday_df = current_df[(current_df.ds >= yesterday) & (current_df.ds < current_ds - pd.Timedelta(hours=current_ds.hour))
].set_index('ds')

In [12]:
yesterday_df.loc[yesterday_df.idxmax().TRG_HALLWAY1]

INFLOW_STRETCHER         5
Infl_Stretcher_cum     124
INFLOW_AMBULATORY        5
Infl_Ambulatory_cum     77
Inflow_Total            10
Inflow_Cum_Total       201
INFLOW_AMBULANCES        2
Infl_Ambulances_cum     37
FLS                      0
CUM_ADMREQ              26
CUM_BA1                 29
WAITINGADM              32
TTStr                  123
TRG_HALLWAY1             4
TRG_HALLWAY_TBS          4
reoriented_cum          10
reoriented_cum_MD        7
QTRACK1                  9
RESUS                    5
Pod_T                   50
POD_GREEN               16
POD_GREEN_TBS            3
POD_YELLOW              17
POD_YELLOW_TBS           2
POD_ORANGE              17
POD_ORANGE_TBS           0
POD_CONS_MORE2H          6
POD_IMCONS_MORE4H        1
POD_XRAY_MORE2H          4
POD_CT_MORE2H           10
POST_POD1                7
VERTSTRET               36
RAZ_TBS                 12
RAZ_LAZYBOY              5
RAZ_WAITINGREZ           6
AMBVERT1                21
AMBVERTTBS              17
Q

In [13]:
yesterday_df

,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,CUM_ADMREQ,...,AMBVERT1,AMBVERTTBS,QTrack_TBS,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM
ds,,,,,,,,,,,,,,,,,,,,,
2023-11-30 23:00:00,7,153,3,93,10,246,2,47,0,38,...,20,15,3,0,9,1,1,4,17,14
2023-11-30 22:00:00,4,146,1,90,5,236,1,45,0,36,...,20,13,3,0,7,0,2,4,19,15
2023-11-30 21:00:00,6,142,4,89,10,231,3,44,0,34,...,31,21,3,0,5,0,3,4,22,16
2023-11-30 20:00:00,6,136,2,85,8,221,3,41,0,32,...,30,24,0,0,8,0,2,3,22,15
2023-11-30 19:00:00,6,130,6,83,12,213,1,38,0,28,...,28,25,0,0,11,0,2,3,20,15
2023-11-30 18:00:00,5,124,5,77,10,201,2,37,0,26,...,21,17,7,0,6,0,2,8,21,15
2023-11-30 17:00:00,13,119,12,72,25,191,2,35,0,24,...,23,23,6,0,9,0,1,9,22,14


In [74]:
yesterday_peaks_df = current_df.set_index('ds').resample("D").max().loc[yesterday]
yesterday_peaks_df

INFLOW_STRETCHER        16
Infl_Stretcher_cum     118
INFLOW_AMBULATORY       15
Infl_Ambulatory_cum    143
Inflow_Total            24
Inflow_Cum_Total       261
INFLOW_AMBULANCES        5
Infl_Ambulances_cum     42
FLS                      2
CUM_ADMREQ              41
CUM_BA1                 30
WAITINGADM              47
TTStr                  127
TRG_HALLWAY1             9
TRG_HALLWAY_TBS          9
reoriented_cum          31
reoriented_cum_MD       17
QTRACK1                 10
RESUS                    6
Pod_T                   55
POD_GREEN               22
POD_GREEN_TBS            5
POD_YELLOW              18
POD_YELLOW_TBS           6
POD_ORANGE              18
POD_ORANGE_TBS           2
POD_CONS_MORE2H         14
POD_IMCONS_MORE4H        3
POD_XRAY_MORE2H          1
POD_CT_MORE2H            4
POST_POD1               13
VERTSTRET               34
RAZ_TBS                  8
RAZ_LAZYBOY             11
RAZ_WAITINGREZ          12
AMBVERT1                44
AMBVERTTBS              34
Q

In [76]:
yesterday_peak_prepod = yesterday_peaks_df.TRG_HALLWAY1
yesterday_peak_prepod

9

In [77]:
yesterday_peak_family_room = yesterday_peaks_df.POST_POD1
yesterday_peak_family_room

13

In [ ]:
myTeamsMessage.payload = {
    "@type": "MessageCard",
    "@context": "http://schema.org/extensions",
    "themeColor": "0076D7",
    "summary": "Morning Report",
    "sections": [{
        "activityTitle": "Morning Report",
        "activitySubtitle": "On Project Tango",
        "activityImage": "https://adaptivecards.io/content/cats/3.png",
        "facts": alerts_for_teams,
        "markdown": True
    }],
}

myTeamsMessage.printme()

myTeamsMessage.send()

In [20]:
myTeamsMessage.payload = {
    "@type": "MessageCard",
    "@context": "https://schema.org/extensions",
    "summary": "Summary",
    "title": "Connector Card Markdown formatting",
    "sections": [
        {
            "text": "This is some **bold** text"
        },
        {
            "text": "This is some _italic_ text"
        },
        {
            "text": "# Header 1\r## Header 2\r### Header 3"
        },
        {
            "text": "- Bullet \r- List \r"
        },
        {
            "text": "1. Numbered\r1. List \r"
        },
        {
            "text": "Link: [Bing](https://www.bing.com)"
        },
        {
            "text": "embedded image link: ![Duck on a rock](https://aka.ms/Fo983c)"
        },
        {
            "text": "`preformatted text`"
        },
        {
            "text": "Newlines (backslash n, backslash n):\n\nline a\n\nline b\n\nline c"
        },
        {
            "text": ">This is a blockquote"
        }
     ]
  }


myTeamsMessage.printme()
myTeamsMessage.send()

hookurl: https://mcgill.webhook.office.com/webhookb2/99b2c50a-f950-46a1-9c52-1467b78e2d13@cd319671-52e7-4a68-afa9-fcf8f89f09ea/IncomingWebhook/0ca0751461fe470e975fc90b5fe59370/509fb105-b979-47ab-b030-d393cc03dbf9
payload: {'@type': 'MessageCard', '@context': 'https://schema.org/extensions', 'summary': 'Summary', 'title': 'Connector Card Markdown formatting', 'sections': [{'text': 'This is some **bold** text'}, {'text': 'This is some _italic_ text'}, {'text': '# Header 1\r## Header 2\r### Header 3'}, {'text': '- Bullet \r- List \r'}, {'text': '1. Numbered\r1. List \r'}, {'text': 'Link: [Bing](https://www.bing.com)'}, {'text': 'embedded image link: ![Duck on a rock](https://aka.ms/Fo983c)'}, {'text': '`preformatted text`'}, {'text': 'Newlines (backslash n, backslash n):\n\nline a\n\nline b\n\nline c'}, {'text': '>This is a blockquote'}]}


In [25]:
stats = {'yesterday_total_inflow':300, 'yesterday_ambulances':50, 'yesterday_reorientations':40}

morningReport = """
# Yesterday
Inflow: {yesterday_total_inflow} \n
Ambulances: {yesterday_ambulances} \n
Reorientations: {yesterday_reorientations} \n
""".format(**stats)

myTeamsMessage.payload = {
    "@type": "MessageCard",
    "@context": "https://schema.org/extensions",
    "summary": "Summary",
    "title": "Morning Report",
    "sections": [
        {"text": morningReport}
     ]
  }


myTeamsMessage.printme()
myTeamsMessage.send()

hookurl: https://mcgill.webhook.office.com/webhookb2/99b2c50a-f950-46a1-9c52-1467b78e2d13@cd319671-52e7-4a68-afa9-fcf8f89f09ea/IncomingWebhook/0ca0751461fe470e975fc90b5fe59370/509fb105-b979-47ab-b030-d393cc03dbf9
payload: {'@type': 'MessageCard', '@context': 'https://schema.org/extensions', 'summary': 'Summary', 'title': 'Morning Report', 'sections': [{'text': '\n# Yesterday\nInflow: 300 \n\nAmbulances: 50 \n\nReorientations: 40 \n\n'}]}


True